In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import pandas as pd
import os,sys

In [ ]:
#Opening and loading the json 
f = open('../input/result2/result2.json', encoding='utf8')
data = json.load(f)

In [ ]:
#Retreive only the messages tag from json
msgs = data['messages']
dmain = pd.DataFrame(msgs)

In [ ]:
#Create a new dataframe with only required columns (id,from,reply_to_message_id and text)
df_new = dmain.filter(items = ['id','from', 'reply_to_message_id' ,'text'])

In [ ]:
df_new['reply_to_message_id'] = df_new['reply_to_message_id'].fillna(0)
df_new

In [ ]:
#Retrieve all the answers by Tim and Raj
#Retrieve only if the length of text is greater than 8 and if it is a reply to another message, basically chk if 
#reply_to_message_id > 0
df_ans = df_new.loc[df_new['from'].isin(['Tim','Raj']) & (df_new['text'].str.len() > 8) & 
                    (df_new["reply_to_message_id"] > 0), ['id','from', 'reply_to_message_id' ,'text']]


#List of words to be searched in the df_ans to eliminate those answers
dropList = ['how','when','which','what','good morning','good evening','good afternoon','awesome','click',
            'thanks','share','introduce','welcome','great','unfortunately',
            'congratulations', 'congrats','wow','wife','makhan']

#Eliminate all messages which has above words
df_ans = df_ans[~df_ans['text'].str.lower().str.contains('|'.join(dropList),na=False)]
# 

#eliminate "?"
df_ans = df_ans[~df_ans['text'].str.contains("?", regex=False)]

In [ ]:
#Retrieve all the questions
#List of words to be searched to retrieve qtns asked to Tim and Raj
include_list =['tell','can','what','where','when','how','which','who','why','suggest','take']

df_ques = df_new.loc[~df_new['from'].isin(['Tim','Raj']) & 
                     (df_new['text'].str.lower().str.contains('|'.join(include_list),na=False)) , 
                     ['id','from', 'reply_to_message_id' ,'text']]

#List of words to be searched to remove unwanted qtns
remove_list =['pin','great','wife','mrs','joined','whatsapp','thanks','good morning','good evening', 
              'gud nite', 'good night' 'good afternoon','dcotor','meant','hope','doctor','feature',
              'qualification','forward']
df_ques = df_ques[~df_ques['text'].str.lower().str.contains('|'.join(remove_list),na=False)]

In [ ]:
#merge both the question and answer dataframe with reply_to_message_id from answer DF and Id col  
qtns = pd.merge(df_ques, df_ans, left_on='id', right_on='reply_to_message_id')
# qtns

#Extract only the required columns from the merged o/p
qtns[['id_x','from_x','text_x','id_y','from_y','reply_to_message_id_y','text_y']]

In [ ]:
#using for loop
#extracting only 2 cols Qs and As(text_x = question and text_y = answer) and loop thru and print out Q and A

qtns = qtns[['text_x','text_y']]

f = open(r'ED_QA.txt','a', encoding='utf8'); 
sys.stdout=f

for index, row in qtns.iterrows():
    print("Question : " + str(row['text_x']))
    print("Answer : " + str(row['text_y']))    
